In [107]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [108]:
df = pd.read_csv("/content/drive/MyDrive/diamonds/diamonds_improvement_3.csv", index_col=0)
df

,carat,depth,table,price,x,y,z,cut_Ideal,cut_Premium,cut_Good,...,color_D,clarity_SI2,clarity_SI1,clarity_VS1,clarity_VS2,clarity_VVS2,clarity_VVS1,clarity_I1,clarity_IF,density
0,0.23,61.5,55.0,326,3.95,3.98,2.43,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.006021
1,0.21,59.8,61.0,326,3.89,3.84,2.31,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.006086
2,0.23,56.9,65.0,327,4.05,4.07,2.31,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.006040
3,0.29,62.4,58.0,334,4.20,4.23,2.63,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.006207
4,0.31,63.3,58.0,335,4.34,4.35,2.75,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.005971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53761,0.72,60.8,57.0,2757,5.75,5.76,3.50,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.006211
53762,0.72,63.1,55.0,2757,5.69,5.75,3.61,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.006096
53763,0.70,62.8,60.0,2757,5.66,5.68,3.56,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.006116
53764,0.86,61.0,58.0,2757,6.15,6.12,3.74,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.006109


In [109]:
X = df.drop(["price"], axis=1)
y = df["price"]

In [110]:
X.shape, y.shape

((53766, 27), (53766,))

In [111]:
num_cols= ["carat", "depth", "table", "density", "x", "y", "z"]

In [112]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X[num_cols])
X[num_cols] = sc.transform(X[num_cols])

In [113]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    shuffle=True,
                                                    test_size=.2,
                                                    random_state=0)

In [114]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((43012, 27), (10754, 27), (43012,), (10754,))

In [115]:
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

In [116]:
params = {
    "objective": "mae",
    "boosting": "gbdt",
    "verbose": -1,
    "learning_rate": .01
}

In [117]:
kf = KFold(n_splits=5, shuffle=True, random_state=0)

valid_score = []
models = []
oof = np.zeros(len(X_train))



In [118]:
for fold, (tr_idx, va_idx) in enumerate(kf.split(X=X_train, y=y_train)):
    X_tr = X_train.iloc[tr_idx, :]
    y_tr = y_train.iloc[tr_idx]
    X_va = X_train.iloc[va_idx, :]
    y_va = y_train.iloc[va_idx]

    lgb_train = lgb.Dataset(X_tr, y_tr)
    lgb_eval = lgb.Dataset(X_va, y_va, reference=lgb_train)

    model = lgb.train(params,
                      train_set=lgb_train,
                      num_boost_round=10000,
                      valid_sets=[lgb_train, lgb_eval],
                      valid_names=["train", "valid"],
                      callbacks=[lgb.early_stopping(100),
                                 lgb.log_evaluation(500)])

    y_va_pred = model.predict(X_va, num_iteration=model.best_iteration)
    score = mean_absolute_error(y_va, y_va_pred)
    print(f"fold:{fold+1}, MAE:{score:.3f}")
    print("-------------------------------------------------------------")

    valid_score.append(score)
    models.append(model)
    oof[va_idx] = y_va_pred






Training until validation scores don't improve for 100 rounds
[500]	train's l1: 348.718	valid's l1: 347.086
[1000]	train's l1: 282.617	valid's l1: 283.937
[1500]	train's l1: 265.249	valid's l1: 271.097
[2000]	train's l1: 254.927	valid's l1: 265.475
[2500]	train's l1: 248.895	valid's l1: 262.827
[3000]	train's l1: 243.835	valid's l1: 260.935
[3500]	train's l1: 238.516	valid's l1: 259.775
[4000]	train's l1: 235.043	valid's l1: 259.261
[4500]	train's l1: 231.729	valid's l1: 258.579
[5000]	train's l1: 229.975	valid's l1: 258.32
[5500]	train's l1: 227.985	valid's l1: 257.953
[6000]	train's l1: 226.052	valid's l1: 257.61
[6500]	train's l1: 224.113	valid's l1: 257.205
[7000]	train's l1: 222.024	valid's l1: 257.023
Early stopping, best iteration is:
[6920]	train's l1: 222.153	valid's l1: 257.01
fold:1, MAE:257.010
-------------------------------------------------------------
Training until validation scores don't improve for 100 rounds
[500]	train's l1: 347.41	valid's l1: 351.233
[1000]	train'

In [119]:
cv_score = np.mean(valid_score)
print(f"cv score: {cv_score:.3f}")

cv score: 266.387
